In [40]:
#!pip install -U pyLDAvis
#!pip install -U pandas

In [41]:
## imports
import os, sys
import pprint as pp
import unicodedata

In [42]:
## 一つ上の階層のファイルを見るように設定
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))

In [43]:
## target language
## a key must be part of a file name 
target_lang_dict = {    'en_US' : 'English (US)',
                        'en_UK' : 'English (UK)',
                        'en_N_only' : 'English noun (WN)',
                        'en_V_only' : 'English verb (WN)',
                        'en_A_only' : 'English adj (WN)',
                        'en_R_only' : 'English adv (WN)',
                        'ar'    : 'Arabic',
                        'de'    : 'German',
                        'de_N_only' : 'German Nouns',
                        'de_non_N_only' : 'German Non-nouns',
                        'eo'    : 'Esperanto',
                        'es_ES' : 'Spanish (Spain)',
                        'es_MX' : 'Spanish (Mexico)',
                        'fi'    : 'Finnish',
                        'fr_FR' : 'French (France)',
                        'fr_QC' : 'French (Quebec)',
                        'is'    : 'Icelandic',
                        'ir'    : 'Irish',
                        'nl'    : 'Dutch',
                        'ro'    : 'Romanian',
                        'sw'    : 'Swahili' }
## proper language selection
target_lang_keys = [    'en_US', # 0
                        'en_UK', # 1
                        'en_N_only', # 2
                        'en_V_only', # 3
                        'en_A_only', # 4
                        'en_R_only', # 5
                        'ar', # 6
                        'de', # 7
                        'de_N_only', # 8
                        'de_non_N_only', # 9
                        'eo', 'es_ES', 'es_MX',
                        'fi', 'fr_FR', 'fr_QC',
                        'is', 'nl', 'ro', 'sw',
                        'ir' # This lacks sound
                    ]
## check
target_lang_key  = target_lang_keys[3]
print(f"target_lang_key: {target_lang_key}")
print(f"target lang: {target_lang_dict[target_lang_key]} [{target_lang_key}]")
## target_attr [effective only for Irish]
target_class = ""
#target_class = None # This causes an unrediable error
if target_lang_key == "ir":
    target_classes = [ 'adjectives', 'nouns', 'verbs' ]
    target_class = f"-{target_classes[3]}"
print(f"target_class: {target_class}")

target_lang_key: en_V_only
target lang: English verb (WN) [en_V_only]
target_class: 


In [44]:
## LDA/HDP
apply_term_filtering = True
## The following parameters need to be relatively large to prevent "Row sum not equal 1" error
term_minfreq         = 2
abuse_threshold      = 0.05 # larger value selects shorter units, smaller value selects longer units
min_bot_size         = 3
# number of terms listed for a given topic
n_terms_to_show = 60

In [45]:
## term settings
term_classes        = [ 'spell', 'sound' ]
term_class          = term_classes[1]
ngram_is_inclusive  = True
## doc settings
max_doc_size        = 11
min_doc_size        =  3
print(f"max_doc_size: {max_doc_size}")
print(f"min_doc_size: {min_doc_size}")
### boundary handling
add_boundary       = True
boundary_mark      = "#"
## term setting
gap_marker           = "…"
term_is_skippy     = True
n_for_ngram        = 4
max_gap_ratio      = 1.0
max_gap_size       = round(max_doc_size * max_gap_ratio)
print(f"term_class: {term_class}")
print(f"term_is_skippy: {term_is_skippy}")
print(f"max_gap_size: {max_gap_size}")
print(f"n_for_ngram: {n_for_ngram}")
### accent handling
suppress_accents   = True
accent_marks       = [ "ˈ", "ˌ" ] 
if term_class == 'sound':
    if suppress_accents:
        accent_status = "-unaccented"
    else:
        accent_stratus = "-accented"
else:
    accent_status = ""
print(f"accent_status: {accent_status}")
## define term_type
if term_class == 'spell':
    if term_is_skippy:
        term_type = f"sp_skippy{n_for_ngram}gram"
    else:
        term_type = f"sp_{n_for_ngram}gram"
else:
    if term_is_skippy:
        term_type = f"sn_skippy{n_for_ngram}gram"
    else:
        term_type = f"sn_{n_for_ngram}gram"
## check
print(f"term_type: {term_type}")

max_doc_size: 11
min_doc_size: 3
term_class: sound
term_is_skippy: True
max_gap_size: 11
n_for_ngram: 4
accent_status: -unaccented
term_type: sn_skippy4gram


In [46]:
## sampling
source_sampling          = True
source_sampling_rate     = 0.5
source_sampling_max_size = 5000
second_sampling          = False
second_sampling_rate     = 0.7

In [47]:
## set target files
import glob
data_dir1     = "data/open-dict-ipa/data1"
data_dir2     = "data/open-dict-ipa/data1a"
data_dir3     = "data/wn3"
data_dir4     = "data/irish"
target_files = glob.glob(f"{data_dir1}/*")
target_files2 = glob.glob(f"{data_dir2}/*")
target_files.extend(target_files2)
target_files3 = glob.glob(f"{data_dir3}/*")
target_files.extend(target_files3)
target_files4 = glob.glob(f"{data_dir4}/*")
target_files.extend(target_files4)
#
target_files = sorted([ file for file in target_files if ".csv" in file ])
pp.pprint(target_files)

['data/irish/word-irish-adjectives-spell.csv',
 'data/irish/word-irish-noun-phrases-spell.csv',
 'data/irish/word-irish-nouns-spell.csv',
 'data/irish/word-irish-possessives-spell.csv',
 'data/irish/word-irish-prepositions-spell.csv',
 'data/irish/word-irish-verbs-spell.csv',
 'data/open-dict-ipa/data1/ar.csv.gz',
 'data/open-dict-ipa/data1/de.csv.gz',
 'data/open-dict-ipa/data1/en_UK.csv.gz',
 'data/open-dict-ipa/data1/en_US.csv.gz',
 'data/open-dict-ipa/data1/eo.csv.gz',
 'data/open-dict-ipa/data1/es_ES.csv.gz',
 'data/open-dict-ipa/data1/es_MX.csv.gz',
 'data/open-dict-ipa/data1/fa.csv.gz',
 'data/open-dict-ipa/data1/fi.csv.gz',
 'data/open-dict-ipa/data1/fr_FR.csv.gz',
 'data/open-dict-ipa/data1/fr_QC.csv.gz',
 'data/open-dict-ipa/data1/is.csv.gz',
 'data/open-dict-ipa/data1/ja.csv.gz',
 'data/open-dict-ipa/data1/jam.csv.gz',
 'data/open-dict-ipa/data1/ma.csv.gz',
 'data/open-dict-ipa/data1/nb.csv.gz',
 'data/open-dict-ipa/data1/nl.csv.gz',
 'data/open-dict-ipa/data1/or.csv.gz',
 '

In [48]:
#print(target_files)

In [49]:
## get source data from files
import pandas as pd
import gzip
#target_language_key = "en_US" # can be changed to get other languages
#if target_class != "" or target_class is not None:
if target_class != "":
    target_file = [ f for f in target_files if target_lang_key in f and target_class in f ][0]
else:
    target_file = [ f for f in target_files if target_lang_key in f ][0]
print(f"processing: {target_file}")
##
if target_lang_key == "ir":
    col_names = ['spell', 'POS']
else:
    col_names = ['spell', 'sound']
#
if target_file.endswith(".gz"):
    with gzip.open(target_file, "rt") as f:
        raw_df = pd.read_csv(f, encoding = 'utf8', header = None, names = col_names )
else:
    with open(target_file, "rt") as f:
        raw_df = pd.read_csv(f, encoding = 'utf8', header = None, names = col_names )
## normalize characters
raw_df['spell'] = raw_df['spell'].apply(lambda x: unicodedata.normalize('NFC', str(x)))
## modify sound
try:
    sounds = raw_df['sound'].apply(lambda x: x.strip('/') )
    sounds = [ x.split("/,")[0] for x in sounds ] # picks up only the first of multiple entries
    raw_df['sound'] = sounds
except (AttributeError, KeyError):
    pass
#
raw_df.sample(10)

processing: data/wn3/en_V_only.csv


,spell,sound
813,commission,kəˈmɪʃən
4377,torpedo,tɔɹˈpiˌdoʊ
4025,spud,ˈspəd
4415,trap,ˈtɹæp
3541,rhyme,ˈɹaɪm
2022,hex,ˈhɛks
117,anger,ˈæŋɡɝ
2232,interleave,ˌɪntɝˈɫiv
1938,grub,ˈɡɹəb
2959,pig,ˈpɪɡ


In [50]:
## source sampling
len(raw_df)
if source_sampling:
	print(f"source sampling applied")
	if len(raw_df) >= source_sampling_max_size:
		raw_df = raw_df.sample(source_sampling_max_size)
	else:
		raw_df = raw_df.sample(round(len(raw_df) * source_sampling_rate))
## remove accent marking
if suppress_accents:
	try:
		raw_df['sound'] = raw_df['sound'].apply(lambda x: "".join([ y for y in list(x) if y not in accent_marks ]))
	except KeyError:
		pass
## add boudary marks
if add_boundary:
	raw_df['spell'] = raw_df['spell'].apply(lambda x: f"{boundary_mark}{x}{boundary_mark}")
	try:
		raw_df['sound'] = raw_df['sound'].apply(lambda x: f"{boundary_mark}{x}{boundary_mark}")
	except KeyError:
		pass
#
print(raw_df)

source sampling applied
            spell        sound
1914      #graze#      #ɡɹeɪz#
2734     #occult#      #əkəɫt#
4254    #syringe#    #sɪɹɪndʒ#
2907      #pedal#      #pɛdəɫ#
2433   #localize#  #ɫoʊkəɫaɪz#
...           ...          ...
4703  #wisecrack#   #waɪzkɹæk#
1032       #cull#        #kəɫ#
59        #admit#      #ədmɪt#
3729  #serialize#  #sɪɹiəɫaɪz#
2777     #output#     #aʊtpʊt#

[2374 rows x 2 columns]


In [51]:
## generate 1-grams for spell and sound
## spell
raw_df['sp_1gram'] = raw_df['spell'].apply(lambda x: list(str(x)))
# add column of size
raw_df['sp_size'] = raw_df['sp_1gram'].apply(lambda x: len(x))
# add column of count of '-' inside
raw_df['hyphen'] = raw_df['sp_1gram'].apply(lambda x: list(x).count("-"))
# add column of count of '.' inside
raw_df['period'] = raw_df['sp_1gram'].apply(lambda x: list(x).count("."))
## sound
# takes the first entry, removes '/' around
try:
    raw_df['sn_1gram'] = raw_df['sound'].apply(lambda x: list(x) )
except (TypeError, KeyError):
    pass
# add column of size
try:
    raw_df['sn_size'] = raw_df['sn_1gram'].apply(lambda x: len(x))
except KeyError:
    pass
## check
raw_df

,spell,sound,sp_1gram,sp_size,hyphen,period,sn_1gram,sn_size
1914,#graze#,#ɡɹeɪz#,"[#, g, r, a, z, e, #]",7,0,0,"[#, ɡ, ɹ, e, ɪ, z, #]",7
2734,#occult#,#əkəɫt#,"[#, o, c, c, u, l, t, #]",8,0,0,"[#, ə, k, ə, ɫ, t, #]",7
4254,#syringe#,#sɪɹɪndʒ#,"[#, s, y, r, i, n, g, e, #]",9,0,0,"[#, s, ɪ, ɹ, ɪ, n, d, ʒ, #]",9
2907,#pedal#,#pɛdəɫ#,"[#, p, e, d, a, l, #]",7,0,0,"[#, p, ɛ, d, ə, ɫ, #]",7
2433,#localize#,#ɫoʊkəɫaɪz#,"[#, l, o, c, a, l, i, z, e, #]",10,0,0,"[#, ɫ, o, ʊ, k, ə, ɫ, a, ɪ, z, #]",11
...,...,...,...,...,...,...,...,...
4703,#wisecrack#,#waɪzkɹæk#,"[#, w, i, s, e, c, r, a, c, k, #]",11,0,0,"[#, w, a, ɪ, z, k, ɹ, æ, k, #]",10
1032,#cull#,#kəɫ#,"[#, c, u, l, l, #]",6,0,0,"[#, k, ə, ɫ, #]",5
59,#admit#,#ədmɪt#,"[#, a, d, m, i, t, #]",7,0,0,"[#, ə, d, m, ɪ, t, #]",7
3729,#serialize#,#sɪɹiəɫaɪz#,"[#, s, e, r, i, a, l, i, z, e, #]",11,0,0,"[#, s, ɪ, ɹ, i, ə, ɫ, a, ɪ, z, #]",11


In [52]:
## filtering raw_data by size
print(f"term_type: {term_type}")
if "sp_" in term_type:
    df_filtered = raw_df[ (raw_df['sp_size'] <= max_doc_size) & (raw_df['sp_size'] >= min_doc_size) & (raw_df['hyphen'] == 0) & (raw_df['period'] == 0) ]
else:
    df_filtered = raw_df[ (raw_df['sn_size'] <= max_doc_size) & (raw_df['sn_size'] >= min_doc_size) ]
#
df_filtered

term_type: sn_skippy4gram


,spell,sound,sp_1gram,sp_size,hyphen,period,sn_1gram,sn_size
1914,#graze#,#ɡɹeɪz#,"[#, g, r, a, z, e, #]",7,0,0,"[#, ɡ, ɹ, e, ɪ, z, #]",7
2734,#occult#,#əkəɫt#,"[#, o, c, c, u, l, t, #]",8,0,0,"[#, ə, k, ə, ɫ, t, #]",7
4254,#syringe#,#sɪɹɪndʒ#,"[#, s, y, r, i, n, g, e, #]",9,0,0,"[#, s, ɪ, ɹ, ɪ, n, d, ʒ, #]",9
2907,#pedal#,#pɛdəɫ#,"[#, p, e, d, a, l, #]",7,0,0,"[#, p, ɛ, d, ə, ɫ, #]",7
2433,#localize#,#ɫoʊkəɫaɪz#,"[#, l, o, c, a, l, i, z, e, #]",10,0,0,"[#, ɫ, o, ʊ, k, ə, ɫ, a, ɪ, z, #]",11
...,...,...,...,...,...,...,...,...
4703,#wisecrack#,#waɪzkɹæk#,"[#, w, i, s, e, c, r, a, c, k, #]",11,0,0,"[#, w, a, ɪ, z, k, ɹ, æ, k, #]",10
1032,#cull#,#kəɫ#,"[#, c, u, l, l, #]",6,0,0,"[#, k, ə, ɫ, #]",5
59,#admit#,#ədmɪt#,"[#, a, d, m, i, t, #]",7,0,0,"[#, ə, d, m, ɪ, t, #]",7
3729,#serialize#,#sɪɹiəɫaɪz#,"[#, s, e, r, i, a, l, i, z, e, #]",11,0,0,"[#, s, ɪ, ɹ, i, ə, ɫ, a, ɪ, z, #]",11


In [53]:
## define df after second sampling if any
len(df_filtered)
if second_sampling:
    df = df_filtered.sample(round(len(df_filtered) * second_sampling_rate))
else:
    df = df_filtered
len(df)

2221

In [54]:
## spell 2grams
#import ngrams
import gen_ngrams
module_name = "gen_ngrams"
reload_module = False
if reload_module:
    import importlib
    importlib.reload(module_name)

if term_class == 'spell':
    #sp_2grams = [ ngrams.list_gen_ngrams (x, n = 2, check = False) for x in df['sp_1gram'] ]
    sp_2grams = [ gen_ngrams.gen_ngrams (x, n = 2, sep = "", check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_2grams):
            g.extend(list(df['sp_1gram'])[i])
    ## add sp_2gram
    df['sp_2gram'] = sp_2grams

In [55]:
## spell 3grams
#import ngrams
import gen_ngrams
if n_for_ngram > 2 and term_class == 'spell':
    #sp_3grams = [ ngrams.list_gen_ngrams (x, n = 3, check = False) for x in df['sp_1gram'] ]
    sp_3grams = [ gen_ngrams.gen_ngrams (x, n = 3, sep = "", check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_3grams):
            g.extend(list(df['sp_2gram'])[i])
    ## add sp_3gram
    df['sp_3gram'] = sp_3grams

In [56]:
## spell 4grams
#import ngrams
import gen_ngrams
if n_for_ngram > 3 and term_class == 'spell':
    #sp_4grams = [ ngrams.list_gen_ngrams (x, n = 4, check = False) for x in df['sp_1gram'] ]
    sp_4grams = [ gen_ngrams.gen_ngrams (x, n = 4, sep = "", check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_4grams):
            g.extend(list(df['sp_3gram'])[i])
    ## add sp_4gram
    df['sp_4gram'] = sp_4grams

In [57]:
## spell 5grams
#import ngrams
import gen_ngrams
if n_for_ngram > 4 and term_class == 'spell':
    sp_5grams = [ gen_ngrams.gen_ngrams (x, n = 5, sep = "", check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_5grams):
            g.extend(list(df['sp_4gram'])[i])
    ## add sp_5gram
    df['sp_5gram'] = sp_5grams

In [58]:
## spell skippy 2gram
#import ngrams_skippy
import gen_ngrams
reload_module = False
module_name = "gen_ngrams"
if reload_module:
    import importlib
    importlib.reload(module_name)
#
if term_class == 'spell':
    #sp_skippy2grams = [ ngrams_skippy.gen_skippy2grams(x, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sp_skippy2grams = [ gen_ngrams.gen_skippy_ngrams(x, 2, sep = "", max_distance = max_gap_size, missing_mark = gap_marker, check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_skippy2grams):
            g.extend(list(df['sp_1gram'])[i])
    #
    df['sp_skippy2gram'] = sp_skippy2grams

In [59]:
## spell skippy 3gram
#import ngrams_skippy
import gen_ngrams
if n_for_ngram > 2 and term_class == 'spell':
    #sp_skippy3grams = [ ngrams_skippy.gen_skippy3grams(x, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sp_skippy3grams = [ gen_ngrams.gen_skippy_ngrams(x, 3, sep = "", max_distance = max_gap_size, missing_mark = gap_marker, check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_skippy3grams):
            g.extend(list(df['sp_skippy2gram'])[i])
    #
    df['sp_skippy3gram'] = sp_skippy3grams

In [60]:
## spell skippy 4gram
#import ngrams_skippy
import gen_ngrams
if n_for_ngram > 3 and term_class == 'spell':
    #sp_skippy4grams = [ ngrams_skippy.gen_skippy4grams(x, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sp_skippy4grams = [ gen_ngrams.gen_skippy_ngrams(x, 4, sep = "", max_distance = max_gap_size, missing_mark = gap_marker, check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_skippy4grams):
            g.extend(list(df['sp_skippy3gram'])[i])
    #
    df['sp_skippy4gram'] = sp_skippy4grams

In [61]:
## spell skippy 5gram
import gen_ngrams
if n_for_ngram > 4 and term_class == 'spell':
    sp_skippy5grams = [ gen_ngrams.gen_skippy_ngrams(x, 5, sep = "", max_distance = max_gap_size, missing_mark = gap_marker, check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_skippy5grams):
            g.extend(list(df['sp_skippy4gram'])[i])
    #
    df['sp_skippy5gram'] = sp_skippy5grams

In [62]:
## sound 2grams
#import ngrams
import gen_ngrams
module_name = "gen_ngrams"
reload_module = False
if reload_module:
    import importlib
    importlib.reload(module_name)
#
if term_class == 'sound':
    #sn_2grams = [ ngrams.list_gen_ngrams (x, n = 2, check = False) for x in df['sn_1gram'] ]
    sn_2grams = [ gen_ngrams.gen_ngrams (x, n = 2, sep ="", check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_2grams):
            g.extend(list(df['sn_1gram'])[i])
    ## add sn_2gram
    df['sn_2gram'] = sn_2grams

/var/folders/7s/_syxn0dd45lcngw4yl2kywd40000gn/T/ipykernel_7377/3258973378.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sn_2gram'] = sn_2grams


In [63]:
## sound 3grams
#import ngrams
import gen_ngrams
if n_for_ngram > 2 and term_class == 'sound':
    #sn_3grams = [ ngrams.list_gen_ngrams (x, n = 3, check = False) for x in df['sn_1gram'] ]
    sn_3grams = [ gen_ngrams.gen_ngrams (x, n = 3, sep = "", check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_3grams):
            g.extend(list(df['sn_2gram'])[i])
    ## add sn_3gram
    df['sn_3gram'] = sn_3grams

/var/folders/7s/_syxn0dd45lcngw4yl2kywd40000gn/T/ipykernel_7377/1780955445.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sn_3gram'] = sn_3grams


In [64]:
## sound 4grams
#import ngrams
import gen_ngrams
if n_for_ngram > 3 and term_class == 'sound':
    #sn_4grams = [ ngrams.list_gen_ngrams (x, n = 4, check = False) for x in df['sn_1gram'] ]
    sn_4grams = [ gen_ngrams.gen_ngrams (x, n = 4, sep = "", check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_4grams):
            g.extend(list(df['sn_3gram'])[i])
    ## add sn_4gram
    df['sn_4gram'] = sn_4grams

/var/folders/7s/_syxn0dd45lcngw4yl2kywd40000gn/T/ipykernel_7377/2514665346.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sn_4gram'] = sn_4grams


In [65]:
## sound 5grams
import gen_ngrams
if n_for_ngram > 4 and term_class == 'sound':
    sn_5grams = [ gen_ngrams.gen_ngrams (x, n = 5, sep = "", check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_5grams):
            g.extend(list(df['sn_4gram'])[i])
    ## add sn_4gram
    df['sn_5gram'] = sn_5grams

In [66]:
## sound skippy 2gram
#import ngrams_skippy
import gen_ngrams
if term_class == 'sound':
    #sn_skippy2grams = [ ngrams_skippy.gen_skippy2grams(x, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sn_skippy2grams = [ gen_ngrams.gen_skippy_ngrams(x, n = 2, sep = "", max_distance = max_gap_size, missing_mark = gap_marker, check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_skippy2grams):
            g.extend(list(df['sn_1gram'])[i])
    #
    df['sn_skippy2gram'] = sn_skippy2grams

/var/folders/7s/_syxn0dd45lcngw4yl2kywd40000gn/T/ipykernel_7377/1165772347.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sn_skippy2gram'] = sn_skippy2grams


In [67]:
## sound skippy 3gram
#import ngrams_skippy
import gen_ngrams
if n_for_ngram > 2 and term_class == 'sound':
    #sn_skippy3grams = [ ngrams_skippy.gen_skippy3grams(x, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sn_skippy3grams = [ gen_ngrams.gen_skippy_ngrams(x, n = 3, sep = "", max_distance = max_gap_size, missing_mark = gap_marker, check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_skippy3grams):
            g.extend(list(df['sn_skippy2gram'])[i])
    #
    df['sn_skippy3gram'] = sn_skippy3grams

/var/folders/7s/_syxn0dd45lcngw4yl2kywd40000gn/T/ipykernel_7377/2061915864.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sn_skippy3gram'] = sn_skippy3grams


In [68]:
## sound skippy 4gram
#import ngrams_skippy
import gen_ngrams
if n_for_ngram > 3 and term_class == 'sound':
    #sn_skippy4grams = [ ngrams_skippy.gen_skippy4grams(x, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sn_skippy4grams = [ gen_ngrams.gen_skippy_ngrams(x, n = 4, sep = "", max_distance = max_gap_size, missing_mark = gap_marker, check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_skippy4grams):
            g.extend(list(df['sn_skippy3gram'])[i])
    #
    df['sn_skippy4gram'] = sn_skippy4grams

/var/folders/7s/_syxn0dd45lcngw4yl2kywd40000gn/T/ipykernel_7377/1309257999.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sn_skippy4gram'] = sn_skippy4grams


In [69]:
## sound skippy 5gram
import gen_ngrams
if n_for_ngram > 4 and term_class == 'sound':
    sn_skippy5grams = [ gen_ngrams.gen_skippy_ngrams(x, n = 5, sep = "", max_distance = max_gap_size, missing_mark = gap_marker, check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_skippy5grams):
            g.extend(list(df['sn_skippy4gram'])[i])
    #
    df['sn_skippy5gram'] = sn_skippy5grams

In [70]:
## check df
dropped_vars = [ 'sp_size', 'hyphen', 'period', 'sn_size' ]
if term_class == 'spell':
    extra = [ 'sn_1gram', 'sn_2gram', 'sn_3gram', 'sn_4gram',
             'sn_skippy2gram', 'sn_skippy3gram', 'sn_skippy4gram' ]
    dropped_vars.extend(extra)
    target_vars = [ x for x in df.columns if not x in dropped_vars ]
else:
    extra = [ 'sp_1gram', 'sp_2gram', 'sp_3gram', 'sp_4gram',
             'sp_skippy2gram', 'sp_skippy3gram', 'sp_skippy4gram' ]
    dropped_vars.extend(extra)
    target_vars = [ x for x in df.columns if not x in dropped_vars ]    
#
df[target_vars]

,spell,sound,sn_1gram,sn_2gram,sn_3gram,sn_4gram,sn_skippy2gram,sn_skippy3gram,sn_skippy4gram
1914,#graze#,#ɡɹeɪz#,"[#, ɡ, ɹ, e, ɪ, z, #]","[#ɡ, ɡɹ, ɹe, eɪ, ɪz, z#, #, ɡ, ɹ, e, ɪ, z, #]","[#ɡɹ, ɡɹe, ɹeɪ, eɪz, ɪz#, #ɡ, ɡɹ, ɹe, eɪ, ɪz, ...","[#ɡɹe, ɡɹeɪ, ɹeɪz, eɪz#, #ɡɹ, ɡɹe, ɹeɪ, eɪz, ɪ...","[#ɡ, #…ɹ, #…e, #…ɪ, #…z, #…#, ɡɹ, ɡ…e, ɡ…ɪ, ɡ…...","[#ɡɹ, #ɡ…e, #ɡ…ɪ, #ɡ…z, #ɡ…#, #…ɹe, #…ɹ…ɪ, #…ɹ...","[#ɡɹe, #ɡɹ…ɪ, #ɡɹ…z, #ɡɹ…#, #ɡ…eɪ, #ɡ…e…z, #ɡ…..."
2734,#occult#,#əkəɫt#,"[#, ə, k, ə, ɫ, t, #]","[#ə, ək, kə, əɫ, ɫt, t#, #, ə, k, ə, ɫ, t, #]","[#ək, əkə, kəɫ, əɫt, ɫt#, #ə, ək, kə, əɫ, ɫt, ...","[#əkə, əkəɫ, kəɫt, əɫt#, #ək, əkə, kəɫ, əɫt, ɫ...","[#ə, #…k, #…ə, #…ɫ, #…t, #…#, ək, ə…ə, ə…ɫ, ə…...","[#ək, #ə…ə, #ə…ɫ, #ə…t, #ə…#, #…kə, #…k…ɫ, #…k...","[#əkə, #ək…ɫ, #ək…t, #ək…#, #ə…əɫ, #ə…ə…t, #ə…..."
4254,#syringe#,#sɪɹɪndʒ#,"[#, s, ɪ, ɹ, ɪ, n, d, ʒ, #]","[#s, sɪ, ɪɹ, ɹɪ, ɪn, nd, dʒ, ʒ#, #, s, ɪ, ɹ, ɪ...","[#sɪ, sɪɹ, ɪɹɪ, ɹɪn, ɪnd, ndʒ, dʒ#, #s, sɪ, ɪɹ...","[#sɪɹ, sɪɹɪ, ɪɹɪn, ɹɪnd, ɪndʒ, ndʒ#, #sɪ, sɪɹ,...","[#s, #…ɪ, #…ɹ, #…ɪ, #…n, #…d, #…ʒ, #…#, sɪ, s…...","[#sɪ, #s…ɹ, #s…ɪ, #s…n, #s…d, #s…ʒ, #s…#, #…ɪɹ...","[#sɪɹ, #sɪ…ɪ, #sɪ…n, #sɪ…d, #sɪ…ʒ, #sɪ…#, #s…ɹ..."
2907,#pedal#,#pɛdəɫ#,"[#, p, ɛ, d, ə, ɫ, #]","[#p, pɛ, ɛd, də, əɫ, ɫ#, #, p, ɛ, d, ə, ɫ, #]","[#pɛ, pɛd, ɛdə, dəɫ, əɫ#, #p, pɛ, ɛd, də, əɫ, ...","[#pɛd, pɛdə, ɛdəɫ, dəɫ#, #pɛ, pɛd, ɛdə, dəɫ, ə...","[#p, #…ɛ, #…d, #…ə, #…ɫ, #…#, pɛ, p…d, p…ə, p…...","[#pɛ, #p…d, #p…ə, #p…ɫ, #p…#, #…ɛd, #…ɛ…ə, #…ɛ...","[#pɛd, #pɛ…ə, #pɛ…ɫ, #pɛ…#, #p…də, #p…d…ɫ, #p…..."
2433,#localize#,#ɫoʊkəɫaɪz#,"[#, ɫ, o, ʊ, k, ə, ɫ, a, ɪ, z, #]","[#ɫ, ɫo, oʊ, ʊk, kə, əɫ, ɫa, aɪ, ɪz, z#, #, ɫ,...","[#ɫo, ɫoʊ, oʊk, ʊkə, kəɫ, əɫa, ɫaɪ, aɪz, ɪz#, ...","[#ɫoʊ, ɫoʊk, oʊkə, ʊkəɫ, kəɫa, əɫaɪ, ɫaɪz, aɪz...","[#ɫ, #…o, #…ʊ, #…k, #…ə, #…ɫ, #…a, #…ɪ, #…z, #...","[#ɫo, #ɫ…ʊ, #ɫ…k, #ɫ…ə, #ɫ…ɫ, #ɫ…a, #ɫ…ɪ, #ɫ…z...","[#ɫoʊ, #ɫo…k, #ɫo…ə, #ɫo…ɫ, #ɫo…a, #ɫo…ɪ, #ɫo…..."
...,...,...,...,...,...,...,...,...,...
4703,#wisecrack#,#waɪzkɹæk#,"[#, w, a, ɪ, z, k, ɹ, æ, k, #]","[#w, wa, aɪ, ɪz, zk, kɹ, ɹæ, æk, k#, #, w, a, ...","[#wa, waɪ, aɪz, ɪzk, zkɹ, kɹæ, ɹæk, æk#, #w, w...","[#waɪ, waɪz, aɪzk, ɪzkɹ, zkɹæ, kɹæk, ɹæk#, #wa...","[#w, #…a, #…ɪ, #…z, #…k, #…ɹ, #…æ, #…k, #…#, w...","[#wa, #w…ɪ, #w…z, #w…k, #w…ɹ, #w…æ, #w…k, #w…#...","[#waɪ, #wa…z, #wa…k, #wa…ɹ, #wa…æ, #wa…k, #wa…..."
1032,#cull#,#kəɫ#,"[#, k, ə, ɫ, #]","[#k, kə, əɫ, ɫ#, #, k, ə, ɫ, #]","[#kə, kəɫ, əɫ#, #k, kə, əɫ, ɫ#, #, k, ə, ɫ, #]","[#kəɫ, kəɫ#, #kə, kəɫ, əɫ#, #k, kə, əɫ, ɫ#, #,...","[#k, #…ə, #…ɫ, #…#, kə, k…ɫ, k…#, əɫ, ə…#, ɫ#,...","[#kə, #k…ɫ, #k…#, #…əɫ, #…ə…#, #…ɫ#, kəɫ, kə…#...","[#kəɫ, #kə…#, #k…ɫ#, #…əɫ#, kəɫ#, kəɫ#, #kə, #..."
59,#admit#,#ədmɪt#,"[#, ə, d, m, ɪ, t, #]","[#ə, əd, dm, mɪ, ɪt, t#, #, ə, d, m, ɪ, t, #]","[#əd, ədm, dmɪ, mɪt, ɪt#, #ə, əd, dm, mɪ, ɪt, ...","[#ədm, ədmɪ, dmɪt, mɪt#, #əd, ədm, dmɪ, mɪt, ɪ...","[#ə, #…d, #…m, #…ɪ, #…t, #…#, əd, ə…m, ə…ɪ, ə…...","[#əd, #ə…m, #ə…ɪ, #ə…t, #ə…#, #…dm, #…d…ɪ, #…d...","[#ədm, #əd…ɪ, #əd…t, #əd…#, #ə…mɪ, #ə…m…t, #ə…..."
3729,#serialize#,#sɪɹiəɫaɪz#,"[#, s, ɪ, ɹ, i, ə, ɫ, a, ɪ, z, #]","[#s, sɪ, ɪɹ, ɹi, iə, əɫ, ɫa, aɪ, ɪz, z#, #, s,...","[#sɪ, sɪɹ, ɪɹi, ɹiə, iəɫ, əɫa, ɫaɪ, aɪz, ɪz#, ...","[#sɪɹ, sɪɹi, ɪɹiə, ɹiəɫ, iəɫa, əɫaɪ, ɫaɪz, aɪz...","[#s, #…ɪ, #…ɹ, #…i, #…ə, #…ɫ, #…a, #…ɪ, #…z, #...","[#sɪ, #s…ɹ, #s…i, #s…ə, #s…ɫ, #s…a, #s…ɪ, #s…z...","[#sɪɹ, #sɪ…i, #sɪ…ə, #sɪ…ɫ, #sɪ…a, #sɪ…ɪ, #sɪ…..."


In [71]:
## select data type and define doc_dict
import random
if "sp_" in term_type:
    base_type = "spell"
else:
    base_type = "sound"
doc_dict = { i: x for i, x in enumerate(df[base_type]) }
## check
random.sample(doc_dict.items(), 10)

/var/folders/7s/_syxn0dd45lcngw4yl2kywd40000gn/T/ipykernel_7377/1181347170.py:9: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(doc_dict.items(), 10)


[(1594, '#tɪnt#'),
 (1823, '#sɛnsətaɪz#'),
 (1845, '#skɪpɝ#'),
 (1834, '#spɔɪɫ#'),
 (1815, '#jɝn#'),
 (595, '#ɹitɑɹd#'),
 (1975, '#ɛnɝteɪn#'),
 (1269, '#daɪs#'),
 (1221, '#daɪnəmaɪt#'),
 (2023, '#səbvɝt#')]

In [72]:
## select bots for analysis
enable_term_change = False # if you want to change term_type to save time and energy
if enable_term_change:
	term_type = 'sp_skippy4gram'
print(f"(changed) term_type: {term_type}")

## bot stands for 'bag-of-terms', a generalization of 'bag-of-words'
bots = [ x for x in df[term_type] if len(x) > min_bot_size ] # Crucially
import random
random.sample(bots, 1)

(changed) term_type: sn_skippy4gram


[['#tɹæ',
  '#tɹ…p',
  '#tɹ…#',
  '#t…æp',
  '#t…æ…#',
  '#t…p#',
  '#…ɹæp',
  '#…ɹæ…#',
  '#…ɹ…p#',
  '#…æp#',
  'tɹæp',
  'tɹæ…#',
  'tɹ…p#',
  't…æp#',
  'ɹæp#',
  'tɹæp',
  'tɹæ…#',
  'tɹ…p#',
  't…æp#',
  'ɹæp#',
  'ɹæp#',
  '#tɹ',
  '#t…æ',
  '#t…p',
  '#t…#',
  '#…ɹæ',
  '#…ɹ…p',
  '#…ɹ…#',
  '#…æp',
  '#…æ…#',
  '#…p#',
  'tɹæ',
  'tɹ…p',
  'tɹ…#',
  't…æp',
  't…æ…#',
  't…p#',
  'ɹæp',
  'ɹæ…#',
  'ɹ…p#',
  'æp#',
  'tɹæ',
  'tɹ…p',
  'tɹ…#',
  't…æp',
  't…æ…#',
  't…p#',
  'ɹæp',
  'ɹæ…#',
  'ɹ…p#',
  'æp#',
  'ɹæp',
  'ɹæ…#',
  'ɹ…p#',
  'æp#',
  'æp#',
  '#t',
  '#…ɹ',
  '#…æ',
  '#…p',
  '#…#',
  'tɹ',
  't…æ',
  't…p',
  't…#',
  'ɹæ',
  'ɹ…p',
  'ɹ…#',
  'æp',
  'æ…#',
  'p#',
  'tɹ',
  't…æ',
  't…p',
  't…#',
  'ɹæ',
  'ɹ…p',
  'ɹ…#',
  'æp',
  'æ…#',
  'p#',
  'ɹæ',
  'ɹ…p',
  'ɹ…#',
  'æp',
  'æ…#',
  'p#',
  'æp',
  'æ…#',
  'p#',
  'p#',
  '#',
  't',
  'ɹ',
  'æ',
  'p',
  '#']]

In [73]:
## generate dictionary
from gensim.corpora import Dictionary
diction = Dictionary(bots)
print(diction)
#apply_term_filtering = False
if apply_term_filtering:
    print(f"term filtering applied")
    diction.filter_extremes(no_below = term_minfreq, no_above = abuse_threshold)
else:
    print(f"term filtering not applied")
## check
print(diction)
## generate DTM
corpus = [ diction.doc2bow(bot) for bot in bots if len(bot) > min_bot_size ] # Crucially

Dictionary<111226 unique tokens: ['#', '#ɡ', '#ɡɹ', '#ɡɹe', '#ɡɹ…#']...>
term filtering applied
Dictionary<40663 unique tokens: ['#ɡ', '#ɡɹ', '#ɡɹe', '#ɡɹ…#', '#ɡɹ…ɪ']...>


In [74]:
## HDP (n_topics = 15)
import gensim.models
import pyLDAvis.gensim
max_n_topics = 15
hdp15 = gensim.models.HdpModel(corpus, diction, random_state = 1, T = max_n_topics)
vis_data15 = pyLDAvis.gensim.prepare(hdp15, corpus, diction)
pyLDAvis.display(vis_data15)

In [75]:
## save LDAvis output as a html file
save_LDAvis = True
if save_LDAvis:
	lang_dir_name = target_lang_dict[target_lang_key].split()[0]
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data15, vis_output)

In [78]:
## get topic-term matrix
#hdp.get_topics() # =/= show_topics()
#hdp.print_topics()
hdp_topics = hdp.show_topics(num_topics = max_n_topics, num_words = n_terms_to_show,
                               formatted = False)
hdp_dict = { tid: values for tid, values in hdp_topics }

## convert to Pandas dataframe
topics_df = pd.DataFrame.from_dict(hdp_dict)
hdp15_topics_out = "results/terms-by-topics-raw/hdp15_topics.csv"
hdp_df.to_csv(hdp15_topics_out)
## save data
#hdp15_topics.save(hdp15_topics_out)

In [38]:
## topic investigation
import numpy as np
import HDP_helper
reload_module = False
if reload_module:
    import importlib
    importlib.reload(HDP_helper)

documents_topics = np.zeros([hdp15.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in hdp15[c]:
        documents_topics[topic_id][doc_id] = prob

n_docs_to_show = 10
n_terms_to_show = 7
hdp15.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    topic_t = hdp15.print_topic(topic_id, topn = n_terms_to_show)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_show)}")
    print(f"nonzero count: ", len(probs.nonzero()[0]))
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.002 * z# + 0.001 * ɪz + 0.001 * ə…a…# + 0.001 * ɪz# + 0.001 * ə…a + 0.001 * ɪ# + 0.001 * ə…aɪ
nonzero count:  556
	0.9993: #səɫɪdəfaɪ#
	0.9993: #kəstəmaɪz#
	0.9993: #spɛʃəɫaɪz#
	0.9993: #dɹæmətaɪz#
	0.9993: #pæstʃɝaɪz#
	0.9992: #dikəmpoʊz#
	0.9992: #vɪktəmaɪz#
	0.9992: #ɹifaɪnæns#
	0.9992: #dʒəstəfaɪ#
	0.9992: #dɪsəpɔɪnt#
topic_id 1: 0.001 * ə…eɪ + 0.001 * ə…e…# + 0.001 * ə…eɪ…# + 0.001 * ə…ɪt + 0.001 * ə…e + 0.001 * ə…ɪt# + 0.001 * ə…eɪt
nonzero count:  631
	0.9993: #kɑpjəɫeɪt#
	0.9993: #pɑpjəɫeɪt#
	0.9993: #dʒɛməneɪt#
	0.9993: #əsɪməɫeɪt#
	0.9993: #sɛɫəbɹeɪt#
	0.9992: #dɛsəkɹeɪt#
	0.9992: #əsæsəneɪt#
	0.9992: #kɑnsəmeɪt#
	0.9992: #kəmɪsɝeɪt#
	0.9992: #haɪfəneɪt#
topic_id 2: 0.001 * ɹ# + 0.001 * ɹ…k + 0.001 * ɹ…s + 0.001 * ɪk + 0.001 * ɪ…ɫ + 0.001 * s…ɹ + 0.001 * i…ɪ
nonzero count:  656
	0.9993: #kənstɹɪkt#
	0.9992: #kɔɹiəɡɹæf#
	0.9991: #kɑnskɹɪpt#
	0.9991: #əpɹiʃieɪt#
	0.9991: #ɪmeɪʃieɪt#
	0.9991: #ɹitæɫieɪt#
	0.9990: #kɑnstətut#
	0.9990: #ɪɫɛkʃənɪɹ#
	0.9990: #mɪmiəɡɹæf

In [39]:
## reduce terms
reload_modules = False
if reload_modules:
    import importlib
    importlib.reload(term_handlers)

#
check = False
for topic in hdp15_topics:
    if check:
        print(topic)
    topic_id, topic_matrix = topic[0], topic[1]
    terms =  [ x[0] for x in topic_matrix ]
    extended_terms = terms.copy()
    print(f"topic id: {topic_id}")
    print(terms)
    reduced_terms = term_handlers.reduce_by_superposition(terms, min_overlap = 2, check = check)
    #reduced_terms = term_handlers.reduce_by_superposition_under_gap(terms, min_overlap = 2, gap_marker = gap_marker, check = check)
    print(reduced_terms)
    if len(reduced_terms) > 0:
        for term in reduced_terms:
            if not term in extended_terms:
                extended_terms.extend(term)
        extended_terms = sorted(extended_terms, key = lambda x: len(x), reverse = True)
    print(extended_terms)


topic id: 0
['z#', 'ɪz', 'ə…a…#', 'ɪz#', 'ə…a', 'ɪ#', 'ə…aɪ']


NameError: name 'term_handlers' is not defined

In [134]:
## HDP (n_topics = 45)
import gensim.models
import pyLDAvis.gensim
max_n_topics = 45
hdp45 = gensim.models.HdpModel(corpus, diction, random_state = 1, T = max_n_topics)
vis_data45 = pyLDAvis.gensim.prepare(hdp45, corpus, diction)
pyLDAvis.display(vis_data45)

In [135]:
## save LDAvis output as a html file
save_LDAvis = True
if save_LDAvis:
	lang_dir_name = target_lang_dict[target_lang_key].split()[0]
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data45, vis_output)

In [136]:
## topic investigation
import numpy as np
import HDP_helper
documents_topics = np.zeros([hdp45.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in hdp45[c]:
        documents_topics[topic_id][doc_id] = prob
#
n_docs_to_show = 10
n_terms_to_show = 7
hdp45.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    topic_t = hdp.print_topic(topic_id, topn = n_terms_to_show)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_show)}")
    print(f"nonzero count: {len(probs.nonzero()[0])}")
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.002 * ʒ# + 0.002 * dʒ + 0.002 * dʒ# + 0.001 * kə + 0.001 * kə…# + 0.001 * ən# + 0.001 * ʊ#
nonzero count: 306
	0.9992: #ɹidɪskəvɝ#
	0.9992: #dɪsɛɫɝeɪt#
	0.9992: #dɪstʃɑɹdʒ#
	0.9992: #pɹidɛstən#
	0.9992: #kənstɹɪkt#
	0.9992: #ɹiəweɪkən#
	0.9990: #ædvæntɪdʒ#
	0.9988: #kɑnsɝtinə#
	0.9988: #skeɪtbɔɹd#
	0.9988: #dɪstɹəkt#
topic_id 1: 0.004 * z# + 0.003 * ɪz + 0.003 * ɪz# + 0.003 * a…z + 0.003 * a…z# + 0.003 * aɪz + 0.003 * aɪz#
nonzero count: 181
	0.9992: #stɛɹəɫaɪz#
	0.9992: #sɪmbəɫaɪz#
	0.9992: #dʒɛməneɪt#
	0.9992: #dɹæmətaɪz#
	0.9992: #sɪɹiəɫaɪz#
	0.9992: #pætɹənaɪz#
	0.9992: #dʒɝməneɪt#
	0.9992: #faɪnəɫaɪz#
	0.9992: #ɹɛdʒəmənt#
	0.9991: #fɹætɝnaɪz#
topic_id 2: 0.001 * t…t + 0.001 * ʃ# + 0.001 * nd + 0.001 * t…t# + 0.001 * u…# + 0.001 * ʊn + 0.001 * i…t
nonzero count: 212
	0.9992: #kɑnstətut#
	0.9992: #pɹɑstətut#
	0.9991: #səbstətut#
	0.9990: #ədmɪnəstɝ#
	0.9990: #fɔɹɡɹaʊnd#
	0.9990: #dɪsɛmbɑɹk#
	0.9990: #ɡɫeɪsieɪt#
	0.9988: #pɹəpaʊnd#
	0.9987: #ɹiskɛdʒuɫ#
	0.9987: #mɪspɫeɪ

In [137]:
## HDP (n_topics = 90)
import gensim.models
import pyLDAvis.gensim
max_n_topics = 90
hdp90 = gensim.models.HdpModel(corpus, diction, random_state = 1, T = max_n_topics,
                             var_converge = 0.001 # Effective to prevent "Not all row sum equatl to 1" error?
                             )
vis_data90 = pyLDAvis.gensim.prepare(hdp90, corpus, diction)
pyLDAvis.display(vis_data90)

In [138]:
## save LDAvis output as a html file
save_LDAvis = True
if save_LDAvis:
	lang_dir_name = target_lang_dict[target_lang_key].split()[0]
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data90, vis_output)

In [139]:
## topic investigation
import numpy as np
import HDP_helper
documents_topics = np.zeros([hdp90.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in hdp90[c]:
        documents_topics[topic_id][doc_id] = prob
## investigate topics
n_docs_to_show  = 10
n_terms_to_show = 7
hdp90.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    #topic_encoding = ", ".join(hdp.show_topic(topic_id))
    topic_t = hdp.print_topic(topic_id, topn = n_terms_to_show)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_show)}")
    print(f"nonzero count: ", len(probs.nonzero()[0]))
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.003 * ʒ# + 0.003 * dʒ + 0.002 * dʒ# + 0.001 * ɑɹ + 0.001 * ɑɹ…# + 0.001 * kə + 0.001 * kə…#
nonzero count:  195
	0.9992: #ɹidɪskəvɝ#
	0.9992: #dɪsɛɫɝeɪt#
	0.9992: #dɪstʃɑɹdʒ#
	0.9992: #kənstɹɪkt#
	0.9991: #ɹiəweɪkən#
	0.9990: #pɹɪvɪɫədʒ#
	0.9990: #ædvæntɪdʒ#
	0.9988: #kɑnsɝtinə#
	0.9988: #skeɪtbɔɹd#
	0.9988: #dɪstɹəkt#
topic_id 1: 0.005 * z# + 0.005 * ɪz + 0.004 * ɪz# + 0.004 * a…z# + 0.004 * a…z + 0.004 * aɪz + 0.004 * aɪz#
nonzero count:  122
	0.9992: #stɛɹəɫaɪz#
	0.9992: #sɪmbəɫaɪz#
	0.9992: #kɑmjənaɪz#
	0.9992: #dɹæmətaɪz#
	0.9992: #sɪɹiəɫaɪz#
	0.9992: #pætɹənaɪz#
	0.9991: #moʊbəɫaɪz#
	0.9991: #ɔstɹəsaɪz#
	0.9991: #voʊkəɫaɪz#
	0.9991: #fɔɹməɫaɪz#
topic_id 2: 0.002 * ə…ɪt# + 0.002 * ə…ɪt + 0.002 * ə…e…# + 0.002 * ə…e + 0.002 * ə…eɪt + 0.002 * ə…e…t + 0.002 * ə…eɪ
nonzero count:  68
	0.9992: #dʒɛməneɪt#
	0.9991: #dʒɝməneɪt#
	0.9991: #sɪmjəɫeɪt#
	0.9991: #ɹɛdʒəmənt#
	0.9991: #pɹidɛstən#
	0.9991: #ɪtɝpəɫeɪt#
	0.9991: #mæstɝbeɪt#
	0.9991: #pɹɑpəɡeɪt#
	0.9990: #ɛkstɝpeɪt#
	